In [4615]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from dash import Dash,dcc, html, callback,Input,Output,State,callback_context,no_update
import dash_bootstrap_components as dbc
import dash_ag_grid as dag

#### In this small project i want to make analyzing using oop

**Let's start!**

In [4616]:
app = Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True

In [4617]:
df = pd.read_csv("Sample - Superstore.csv",encoding='cp1252') # <- Just reading file

In [4618]:
dfinfo = pd.read_csv('Info.csv')

In [4619]:
df.isna().sum() # We haven't any null here and we can continue work with this project

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [4620]:
df.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [4621]:
df_cat = df.groupby('Category')['Sales'].sum().reset_index()

In [4622]:
df_cat

,Category,Sales
0,Furniture,741999.7953
1,Office Supplies,719047.0320
2,Technology,836154.0330


## Page 0

In [4623]:
df_data = df.copy()

In [4624]:
df_data['Order Date'] = pd.to_datetime(df_data['Order Date'])
df_data = df_data[['Order Date','Sales','Profit']]

In [4625]:
df_data[df_data['Order Date'].between('2014-01-03','2014-01-05')]

,Order Date,Sales,Profit
739,2014-01-04,11.784,4.2717
740,2014-01-04,272.736,-64.7748
741,2014-01-04,3.540,-5.4870
1759,2014-01-05,19.536,4.8840
7980,2014-01-03,16.448,5.5512


In [4626]:
df_data = df_data.groupby('Order Date')[['Sales','Profit']].sum().reset_index()

In [4627]:
df_data['date_num'] = (df_data['Order Date']-pd.to_datetime('2014-01-03')).dt.days

In [4628]:
df_data = df_data.sort_values(by='Order Date')

In [4629]:
df_data.head()

,Order Date,Sales,Profit,date_num
0,2014-01-03,16.448,5.5512,0
1,2014-01-04,288.060,-65.9901,1
2,2014-01-05,19.536,4.8840,2
3,2014-01-06,4407.100,1358.0524,3
4,2014-01-07,87.158,-71.9621,4


In [4630]:
marks = {i : date.strftime(r'%Y-%m-%d') for i,date in zip(df_data['date_num'],df_data['Order Date'])}

In [4631]:
card1 = dbc.Card(
    dbc.CardBody([
        html.H4("Total Sales", className="card-title"),
        html.H2(id='text-card1', className="card-text"),
        html.I(className="bi bi-info-circle-fill me-2"),
    ]),
    id = 'card1',
    color="primary", inverse=True
)

In [4632]:
card2 = dbc.Card(
    dbc.CardBody([
        html.H4("Total Profit", className="card-title"),
        html.H2(id='text-card2', className="card-text"),
        html.I(className="bi bi-info-circle-fill me-2"),
    ]),
    id = 'card2',
    color="primary", inverse=True
)

In [4633]:
range_slider_card1 = dcc.RangeSlider(id = 'range-slider',
                    min=df_data['date_num'].min(),max=df_data['date_num'].max(),step=1,
                    value = [df_data['date_num'].min(),df_data['date_num'].max()],marks=None)

In [4634]:

page_0 = dbc.Container(
    [
    dbc.Row([
    html.H3('Category sales', className='text-left'),
    ]),
dbc.Row([
    dbc.Col(),
    dbc.Col(dcc.RadioItems(
        ['Sales','Profit'],id='sales_or_profit',value='Sales',
        inline=True)),
    ]),
dbc.Row([
    dbc.Col(dcc.Graph(id='categories'),
                  width={"size": 4}),
    dbc.Col(dcc.Graph(id='sub-categories'),width={"size": 4}),
    dbc.Col(dcc.Graph(id='one-sales-profit'),width={"size": 4}),
    ]),
dbc.Row([
    dbc.Col(card1),
    dbc.Col(card2),
    dbc.Col(),
    dbc.Col(),
]),
dbc.Row([
    dbc.Col(range_slider_card1),
]),
dbc.Row([
    dbc.Col(html.Div(id='slider-output', className='mt-4')),
        ]),
   ],fluid=True
   )

## Page 1

In [4635]:
superstore_cities = pd.read_csv("Superstore cities.csv")

In [4636]:
import json
with open('state_codes.json', 'r') as f:
    state_codes = json.load(f)

In [4637]:
df_states_w_city = df.drop('Order Date',axis=1).groupby(['State','City']).sum()[['Sales','Profit']].reset_index()

In [4638]:
df_states_w_city['State code'] = df_states_w_city['State'].map(state_codes)

In [4639]:
df_states_no_city = df_states_w_city.drop('City',axis=1).groupby(['State','State code']).sum().reset_index()

In [4640]:
df_states_no_city.head()

,State,State code,Sales,Profit
0,Alabama,AL,19510.6400,5786.8253
1,Arizona,AZ,35282.0010,-3427.9246
2,Arkansas,AR,11678.1300,4008.6871
3,California,CA,457687.6315,76381.3871
4,Colorado,CO,32108.1180,-6527.8579


In [4641]:
df_states_no_city['Profit'] = df_states_no_city['Profit'].round().apply(lambda x: f'{x} $')

In [4642]:
df_states_no_city['Sales'] = df_states_no_city['Sales'].round()

In [4643]:
mapp = px.choropleth(
    df_states_no_city,
    locations='State code',
    locationmode="USA-states",
    color='Sales',
    scope="usa",
    hover_name='State',
    hover_data={'Sales': True, 'Profit': True},
    color_continuous_scale='Viridis'
)

In [4644]:
page_1 = dbc.Container([
    dbc.Row([
    html.H3('Sales map', className='text-left'),
    ]),
    dbc.Row([
    dbc.Col(dcc.Graph(figure=mapp,id='map',
            style={'padding': '0px', 'margin': '0px','height': '100vh'},
             className='container-fluid'),width=7
    ),
    dbc.Col([
        dbc.Row(
            dcc.Graph(figure = {},id='map_state',
            style={'padding': '0px', 'margin': '0px','height': '45vh'},
            className='container-fluid'),
        style={'height': '50%'},
        ),
        dbc.Row(
        dcc.Graph(figure = {},id='sub-category-in-state',
                 className='container-fluid',
                 style={'padding': '0px', 'margin': '0px','height': '45vh'},),    
        style={'height': '50%'},
        ),
        ]),   
    ]),
],fluid=True, className="mt-1")

In [4645]:
page_2 = dbc.Container([
    dbc.Row([
    html.H3('The most popular products', className='text-left'),
    ]),
    dbc.Row([
        dbc.Col([
            html.H4('Profit by sales'),
            dcc.Dropdown(id='category-for-product',
                         options = [{'label': label, 'value': label} for label in df['Product Name'].unique()],
                         value=['Staples','Staple remover'],
                         multi=True,
                         searchable=True,
                         placeholder="Select products or categories...",
                         clearable=True
                         ),
            dcc.Graph(figure={},id='profit-sale-product-compare')
            ],width=6),
        dbc.Col([
            html.H4('Choose category'),
            dcc.Dropdown(
            options = ['Technology','Furniture','Office Supplies'],
            value = ['Technology','Furniture','Office Supplies'],
            multi = True,
            id = 'category-for-products',    
            ),           
            dcc.Graph(figure = {},id='products'), 
            dcc.Slider(2,15,1,value=7,id='slider-of-products'),          
            ],width=6)
    ]),
],fluid=True, className="mt-4")

In [4646]:
app.layout = dbc.Container([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        brand="Superstore",
        brand_href="/",
        color="primary",
        dark=True,
        children=[
            dbc.NavItem(dcc.Link('Category Sales', href='/', className='nav-link')),
            dbc.NavItem(dcc.Link('Sales map', href='/page-1', className='nav-link')),
            dbc.NavItem(dcc.Link('Products', href='/page-2', className='nav-link')),
        ]
    ),
    html.Div(id='page-content')
], fluid=True)

In [4647]:
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/':
        return page_0
    elif pathname == '/page-1':
        return page_1
    elif pathname == '/page-2':
        return page_2

In [4648]:
@callback(
    Output('sub-categories','figure'),
    Output('one-sales-profit','figure'),
    Output('categories','figure'),
    Input('categories','clickData'),
    Input('sales_or_profit','value'),
    Input('range-slider','value'),
)
def update_bar_char(clickData,val,date):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df_subs_cat = df[df['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    if clickData == None:
         selected_category = 'Furniture'
    else :
        selected_category = clickData['points'][0]['label']       
    filtered_df = df_subs_cat[df_subs_cat['Category'] == selected_category]    
    fig = px.bar(filtered_df.sort_values(by=val), y='Sub-Category', x=val, title=f'{val} in {selected_category}')
    filtered_df2 = filtered_df.copy()
    filtered_df2['coef'] = filtered_df2['Profit']/filtered_df2['Sales']
    fig2 = px.bar(filtered_df2.sort_values(by='coef'),x='Sub-Category',y='coef',title='Profit by one sale')
    fig3 = px.pie(data_frame=df_subs_cat,values=val,names='Category',title='Category')
    return fig,fig2,fig3

In [4649]:
@callback(
    Output('text-card1','children'),
    Output('text-card2','children'),
    Input('range-slider','value')
)
def cards(date):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df_subs_cat = df[df['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    sales = df_subs_cat['Sales'].sum().round()
    profit = df_subs_cat['Profit'].sum().round()
    return f'{sales} $',f'{profit} $'


In [4650]:
@callback(
    Output('slider-output','children'),
    Input('range-slider','value'),
)
def range_slider(val):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[1], unit='D')
    return f'Selected range: {start_date.date()} to {end_date.date()}'

In [4651]:
@app.callback(
    Output('map_state', 'figure'),
    Input('map', 'clickData')
)
def update_bar_chart(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    filtered_df = df_states_w_city[df_states_w_city['State'] == state_name].sort_values(by='Sales')
    fig = px.bar(filtered_df, x='City', y='Sales')
    fig.update_yaxes(range=[0, 10000])
    return fig

In [4652]:
@callback(
    Output('sub-category-in-state','figure'),
    Input('map', 'clickData'),
)
def update_bar_subcategory(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    df_sub_category_state = df[df['State'] == state_name].drop('Order Date',axis=1).\
    groupby(by='Sub-Category').sum().reset_index()[['Sub-Category','Sales']].sort_values(by='Sales')
    print(df_sub_category_state)
    fig = px.bar(df_sub_category_state, x='Sub-Category', y='Sales')
    return fig


In [4653]:
@callback(
Output('products','figure'),
Input('slider-of-products','value'),
Input('category-for-products','value')
)
def product_slider(quantity,category):
    df_ = df[df['Category'].isin(category)]
    df_ = df_['Product Name'].value_counts()[:quantity].reset_index()
    fig = px.pie(df_,values='count',names='Product Name')
    return fig

In [4654]:
@callback(
Output('profit-sale-product-compare','figure'),
Input('category-for-product','value'),
)
def profit_sale_product_compare(selected_product):
     df_ = df[df['Product Name'].isin(selected_product)]
     df_ = df_[['Sales','Profit','Product Name']].groupby('Product Name').sum().reset_index()
     df_['Impact'] = df_['Profit']/df_['Sales']
     df_['Impact'] = df_['Impact'] + 1
     fig = px.bar(df_,x='Product Name',y='Impact')
     return fig


In [4655]:
if __name__ == "__main__":
    app.run_server(debug=True,port=8051)